In [107]:
from future.builtins import next
import re
from numpy import nan
import dedupe
from unidecode import unidecode
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
import json
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType,FloatType
import warnings
warnings.simplefilter(action='ignore')

In [108]:

import os
import csv
import re
import logging
import optparse
import collections

In [109]:
spark = SparkSession \
    .builder \
    .appName("dedupe") \
    .master("local[*]") \
    .getOrCreate()

In [110]:
output_file = 'gazetteer_output.csv'
settings_file = 'gazetteer_learned_settings'
training_file = 'gazetteer_training.json'

In [111]:
primart_key="price"


# canon_file= spark.read.option("header","true").csv("data/AbtBuy_Buy.csv")
# messy_file= spark.read.option("header","true").csv("data/AbtBuy_Abt.csv")

canon_file= "data/AbtBuy_Buy.csv"
messy_file= "data/AbtBuy_Abt.csv"

In [112]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    # column = column.decode("utf8")
    column = unidecode(column)
    column = re.sub('\n', ' ', column)
    column = re.sub('-', '', column)
    column = re.sub('/', ' ', column)
    column = re.sub("'", '', column)
    column = re.sub(",", '', column)
    column = re.sub(":", ' ', column)
    column = re.sub(' +', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column:
        column = None
    return column
def readData(filename):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """
    df= spark.read.option("header","true").csv(filename)

    data_d = {}
    dfn=df.na.fill("")
    results = dfn.toJSON().map(lambda j: json.loads(j)).collect()
    for i, row in enumerate(results):
            clean_row = dict([(k, preProcess(v)) for (k, v) in row.items()])
            if clean_row['price']:
                clean_row['price'] = float(clean_row['price'][1:])
            data_d[filename + str(i)] = dict(clean_row)

    return data_d

In [113]:
messy = readData(messy_file)
canonical = readData(canon_file)

In [114]:
print('N data 2 records: {}'.format(len(canonical)))

print('N data 1 records: {}'.format(len(messy)))

N data 2 records: 1092
N data 1 records: 1081


In [156]:
def descriptions():
        for dataset in (messy, canonical):
            for record in dataset.values():
                yield record['description']

In [165]:
fields = [
            {'field': 'title', 'type': 'String'},
            {'field': 'title', 'type': 'Text', 'corpus': descriptions()},
            {'field': 'description', 'type': 'Text','has missing': True, 'corpus': descriptions()},
            {'field': 'price', 'type': 'Price', 'has missing': True}]

# Create a new gazetteer object and pass our data model to it.
gazetteer = dedupe.Gazetteer(fields)

In [166]:
print('reading labeled examples from ', training_file)
with open(training_file) as tf:
    gazetteer.prepare_training(messy, canonical, training_file=tf)

gazetteer.train()
gazetteer.cleanup_training()

gazetteer.index(canonical)

results = gazetteer.search(messy, n_matches=2, generator=True)

reading labeled examples from  gazetteer_training.json


In [167]:
# training_file = 'gazetteer_training.json'
# with open(training_file) as tf:
#     gazetteer.prepare_training(messy, canonical, training_file=tf)

In [168]:
cluster_membership = {}
cluster_id = 0

for cluster_id, (messy_id, matches) in enumerate(results):
    for canon_id, score in matches:
        cluster_membership[messy_id] = {'Cluster ID': float(cluster_id),
                                        'Link Score': float(score)}
        cluster_membership[canon_id] = {'Cluster ID': float(cluster_id),
                                        'Link Score': float(score)}
        cluster_id += 1

In [184]:
json_cls=[]
for fileno, filename in enumerate((messy_file, canon_file)):
    df= spark.read.option("header","true").csv(filename)
    results = df.toJSON().map(lambda j: json.loads(j)).collect()
    for row_id, row in enumerate(results):
        record_id = filename + str(row_id)
        cluster_details = cluster_membership.get(record_id, {})
        print(cluster_details)
        row['source file'] = fileno
        Dict = {}
        Dict["source_file"] = row['source file']
        
        
        cluster_details.update(Dict)
        cluster_details.update(row)
        json_cls.append(cluster_details)

    

{'Cluster ID': 0.0, 'Link Score': 0.545451819896698, 'unique_id': '1', 'title': 'Linksys EtherFast 8-Port 10/100 Switch - EZXS88W', 'description': 'Linksys EtherFast 8-Port 10/100 Switch - EZXS88W/ 10/100 Dual-Speed Per-Port/ Perfect For Optimizing 10BaseT And 100BaseTX Hardware On The Same Network/ Speeds Of Up To 200Mbps In Full Duplex Operation/ Eliminate Bandwidth Constraints And Clear Up Bottlenecks', 'price': '$44.00', 'source_file': 0, 'source file': 0}
{'Cluster ID': 1.0, 'Link Score': 0.5454487800598145, 'unique_id': '2', 'title': 'Linksys EtherFast10/100 5-Port Auto-Sensing Switch - EZXS55W', 'description': 'Linksys EtherFast10/100 5-Port Auto-Sensing Switch - EZXS55W/ 5 Port 10/100 Autosensing Ports With Both Half And Full Duplex Modes/ Perfect For Integrating Your 10BaseT And 100BaseTX Network Hardware/ Switched 10/100 Ports Run At 10Mbps, 20Mbps, 100Mbps Up To 200Mbps', 'price': '$29.00', 'source_file': 0, 'source file': 0}
{'Cluster ID': 126.0, 'Link Score': 0.54545098543

In [174]:

print(json_cls)

[{'Cluster ID': 0.0, 'Link Score': 0.545451819896698, 'unique_id': '1', 'title': 'Linksys EtherFast 8-Port 10/100 Switch - EZXS88W', 'description': 'Linksys EtherFast 8-Port 10/100 Switch - EZXS88W/ 10/100 Dual-Speed Per-Port/ Perfect For Optimizing 10BaseT And 100BaseTX Hardware On The Same Network/ Speeds Of Up To 200Mbps In Full Duplex Operation/ Eliminate Bandwidth Constraints And Clear Up Bottlenecks', 'price': '$44.00', 'source_file': 0, 'source file': 0}, {'Cluster ID': 1.0, 'Link Score': 0.5454487800598145, 'unique_id': '2', 'title': 'Linksys EtherFast10/100 5-Port Auto-Sensing Switch - EZXS55W', 'description': 'Linksys EtherFast10/100 5-Port Auto-Sensing Switch - EZXS55W/ 5 Port 10/100 Autosensing Ports With Both Half And Full Duplex Modes/ Perfect For Integrating Your 10BaseT And 100BaseTX Network Hardware/ Switched 10/100 Ports Run At 10Mbps, 20Mbps, 100Mbps Up To 200Mbps', 'price': '$29.00', 'source_file': 0, 'source file': 0}, {'Cluster ID': 126.0, 'Link Score': 0.54545098

In [175]:
df1=spark.createDataFrame(json_cls)

In [181]:
df1.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("Untitled Folder/")


In [182]:
df1.count()

2173